In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [2]:
cine21_url = 'http://www.cine21.com/rank/person/'
res = requests.get(cine21_url)
print(res.text) # get으로 할 경우 배우 리스트가 응답으로 반환되지 않음(사이트마다 다름)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
	<meta property="fb:pages" content="1641311652800771" />
	<meta name="google-site-verification" content="vTM0gmeRzJwn1MIM1LMSp3cxP_SaBzch1ziRY255RHw" />
	<meta name="google-site-verification" content="5yOe6b_e_3rr7vNDwgXJw_8wLZQGx4lJ_V48KNPrqkA" />
	<meta name="naver-site-verification" content="20defde86fc4464f2693891567a98905bd0a60d1" />
	<meta name='dailymotion-domain-verification' content='dmds9ks357rhqvdnk' />
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<title>씨네21</title>
	<link rel="stylesheet" type="text/css" media="all" href="/inc/www/css/default1.css" />
	<link rel="stylesheet" type="text/css" media="all" href="/inc/www/css/content1.css" />
	<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.0/jquery.min.js"></scr

### 크롤링 주소 요청
dictionary를 이용하여 form data 생성   
request모듈의 post 함수를 이용하여 요청 전송

- 첫 번째 인자: 요청 url
- 두 번째 인자: form data

In [3]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2022-01'
post_data['gender'] = 'all'
post_data['page'] = 1

res = requests.post(cine21_url, data=post_data)

In [4]:
res.text

'\t\t\t<ul class="people_list">\r\n\t\t\t\t<li class="people_li">\r\n\t\t\t\t\t<a href="/db/person/info/?person_id=78487"><img src="https://image.cine21.com/resize/cine21/still/2017/1207/15_06_46__5a28da76c2e01[X145,145].jpg" alt="" class="people_thumb" target="_blank" /></a>\r\n\t\t\t\t\t<div class="name"><a href="/db/person/info/?person_id=78487">강하늘(2편)</a></div>\r\n\t\t\t\t\t<ul class="num_info">\r\n\t\t\t\t\t\t<li><span class="tit">흥행지수</span><strong>79,188</strong></li>\r\n\t\t\t\t\t\t<!--\r\n\t\t\t\t\t\t<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>\r\n\t\t\t\t\t\t-->\r\n\t\t\t\t\t</ul>\r\n\t\t\t\t\t<!-- 영화포스터는 최대 5개까지만 -->\r\n\t\t\t\t\t<ul class="mov_list">\r\n\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t<li>\r\n\t\t\t\t\t\t\t\t<a href="/movie/info/?movie_id=56540">\r\n\t\t\t\t\t\t\t\t\t<img src="https://image.cine21.com/resize/cine21/poster/2022/0127/56540_61f1fcfdd84ce[X85,120].jpg" alt="" class="thumb"  target="_blank" />\r\n\t\t\t\t\t\t\

In [5]:
soup = BeautifulSoup(res.text, 'html.parser')

### 배우 이름 추출

In [6]:
# select(): find_all()과 동일한 작동을 하는 함수
# 차이점은 파라미터로 css selector를 사용
# 좀 더 간편하다
soup.select('li.people_li div.name') # class가 people_li인 li 태그 중 class가 name인 div 태그 list

[<div class="name"><a href="/db/person/info/?person_id=78487">강하늘(2편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=56311">한효주(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=71308">이광수(2편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=15225">권상우(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=60358">조진웅(2편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=20772">박희순(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=95811">채수빈(1편)</a></div>]

In [7]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print(actor.text)

강하늘(2편)
한효주(1편)
이광수(2편)
권상우(1편)
조진웅(2편)
박희순(1편)
채수빈(1편)


이름만 추출할 수 없을까? -> 정규표현식 사용

In [8]:
test_data = '조우진(2편)'
# re.sub(정규 표현식, 치환 문자, 대상 문자열)
re.sub('\(\w*\)', '', test_data) # \w: 문자, 숫자와 매치

'조우진'

In [9]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print(re.sub('\(\w*\)', '', actor.text))

강하늘
한효주
이광수
권상우
조진웅
박희순
채수빈


### 배우 상세 정보 추출

In [10]:
actors = soup.select('li.people_li div.name')
# select_one(): find()와 대응되는 함수
for actor in actors:
    print (actor.select_one('a'))

<a href="/db/person/info/?person_id=78487">강하늘(2편)</a>
<a href="/db/person/info/?person_id=56311">한효주(1편)</a>
<a href="/db/person/info/?person_id=71308">이광수(2편)</a>
<a href="/db/person/info/?person_id=15225">권상우(1편)</a>
<a href="/db/person/info/?person_id=60358">조진웅(2편)</a>
<a href="/db/person/info/?person_id=20772">박희순(1편)</a>
<a href="/db/person/info/?person_id=95811">채수빈(1편)</a>


In [11]:
actors = soup.select('li.people_li div.name')
actors_info_list = list()

for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href'] # 태그 내 속성 접근
    response_actor = requests.get(actor_link) # 배우 상세 페이지 이동
    soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    print(actor_details)

[<li><span class="tit">다른 이름</span>김하늘</li>, <li><span class="tit">직업</span>배우</li>, <li><span class="tit">생년월일</span>1990-02-21</li>, <li><span class="tit">성별</span>남</li>, <li><span class="tit">홈페이지</span>
<a href="http://weibo.com/galpos3?is_hot=1" target="_blank">http://weibo.com/galpos3?is_hot=1</a><br/>
</li>, <li><span class="tit">신장/체중</span>181cm, 70kg</li>, <li><span class="tit">학교</span>중앙대학교 연극학과</li>]
[<li><span class="tit">직업</span>배우</li>, <li><span class="tit">생년월일</span>1987-02-22</li>, <li><span class="tit">성별</span>여</li>, <li><span class="tit">홈페이지</span>
<a href="https://www.facebook.com/hhj.official" target="_blank">https://www.facebook.com/hhj.official</a><br/>
</li>, <li><span class="tit">신장/체중</span>170cm</li>, <li><span class="tit">학교</span>동국대학교 연극영화</li>, <li><span class="tit">취미</span>영화감상</li>]
[<li><span class="tit">직업</span>배우</li>, <li><span class="tit">생년월일</span>1985-07-14</li>, <li><span class="tit">성별</span>남</li>, <li><span class="tit">홈페이지</span>


배우마다 가지고 있는 정보가 다르다!   
-> dictionary 형태로 저장

In [12]:
actors = soup.select('li.people_li div.name')
actors_info_list = list()

for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href'] # 태그 내 속성 접근
    response_actor = requests.get(actor_link) # 배우 상세 페이지 이동
    soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    actor_info_dict = dict()
    
    for actor_detail in actor_details:
        actor_detail_key = actor_detail.select_one('span.tit').text # 상세정보 key값
        actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail)) # span 태그 제거
        actor_detail_value = re.sub('<.*?>', '', actor_detail_value) # 상세정보 value값
        actor_info_dict[actor_detail_key] = actor_detail_value
        
    actors_info_list.append(actor_info_dict)

actors_info_list

[{'다른 이름': '김하늘',
  '직업': '배우',
  '생년월일': '1990-02-21',
  '성별': '남',
  '홈페이지': '\nhttp://weibo.com/galpos3?is_hot=1\n',
  '신장/체중': '181cm, 70kg',
  '학교': '중앙대학교 연극학과'},
 {'직업': '배우',
  '생년월일': '1987-02-22',
  '성별': '여',
  '홈페이지': '\nhttps://www.facebook.com/hhj.official\n',
  '신장/체중': '170cm',
  '학교': '동국대학교 연극영화',
  '취미': '영화감상'},
 {'직업': '배우',
  '생년월일': '1985-07-14',
  '성별': '남',
  '홈페이지': '\nhttps://twitter.com/masijacoke85\nhttps://www.instagram.com/masijacoke850714/\n',
  '신장/체중': '190cm',
  '소속사': '킹콩엔터테인먼트'},
 {'다른 이름': 'Kwon Sang Woo',
  '직업': '배우',
  '생년월일': '1976-08-05',
  '성별': '남',
  '신장/체중': '183cm, 72kg',
  '학교': '한남대학교 미술교육학 학사',
  '취미': '수영, 헬스, 복싱',
  '특기': '농구',
  '소속사': '벨액터스 엔터테인먼트'},
 {'다른 이름': '조원준',
  '직업': '배우',
  '생년월일': '1976-03-03',
  '성별': '남',
  '홈페이지': '\nhttp://www.facebook.com/saram.chojinwoong\n',
  '신장/체중': '185cm, 98kg',
  '학교': '경성대학교 연극영화',
  '특기': '진도 북춤',
  '소속사': '㈜사람엔터테인먼트'},
 {'직업': '배우',
  '생년월일': '1970-02-13',
  '성별': '남',
  '홈페이지': '\nhttps:

### 최종 코드

In [13]:
actors_info_list = list()
actors = soup.select('li.people_li div.name')

for index, actor in enumerate(actors):
    actor_info_dict = dict()
    actor_name = re.sub('\(\w*\)', '', actor.text)
    actor_info_dict['이름'] = actor_name
    
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    for actor_detail in actor_details:
        actor_detail_key = actor_detail.select_one('span.tit').text
        actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail))
        actor_detail_value = re.sub('<.*?>', '', actor_detail_value)
        actor_info_dict[actor_detail_key] = actor_detail_value
        
    actors_info_list.append(actor_info_dict)

actors_info_list

[{'이름': '강하늘',
  '다른 이름': '김하늘',
  '직업': '배우',
  '생년월일': '1990-02-21',
  '성별': '남',
  '홈페이지': '\nhttp://weibo.com/galpos3?is_hot=1\n',
  '신장/체중': '181cm, 70kg',
  '학교': '중앙대학교 연극학과'},
 {'이름': '한효주',
  '직업': '배우',
  '생년월일': '1987-02-22',
  '성별': '여',
  '홈페이지': '\nhttps://www.facebook.com/hhj.official\n',
  '신장/체중': '170cm',
  '학교': '동국대학교 연극영화',
  '취미': '영화감상'},
 {'이름': '이광수',
  '직업': '배우',
  '생년월일': '1985-07-14',
  '성별': '남',
  '홈페이지': '\nhttps://twitter.com/masijacoke85\nhttps://www.instagram.com/masijacoke850714/\n',
  '신장/체중': '190cm',
  '소속사': '킹콩엔터테인먼트'},
 {'이름': '권상우',
  '다른 이름': 'Kwon Sang Woo',
  '직업': '배우',
  '생년월일': '1976-08-05',
  '성별': '남',
  '신장/체중': '183cm, 72kg',
  '학교': '한남대학교 미술교육학 학사',
  '취미': '수영, 헬스, 복싱',
  '특기': '농구',
  '소속사': '벨액터스 엔터테인먼트'},
 {'이름': '조진웅',
  '다른 이름': '조원준',
  '직업': '배우',
  '생년월일': '1976-03-03',
  '성별': '남',
  '홈페이지': '\nhttp://www.facebook.com/saram.chojinwoong\n',
  '신장/체중': '185cm, 98kg',
  '학교': '경성대학교 연극영화',
  '특기': '진도 북춤',
  '소속사': '㈜사람엔터테인먼

# 과제

1. 흥행지수, 출연 영화, 랭킹 추가(출연 영화 value는 리스트로)
2. 1~10페이지까지 크롤링 범위 확장

In [14]:
cine21_url = 'http://www.cine21.com/rank/person/content'
actors_info_list = list()

for page in range(1,11):
    post_data = dict()
    post_data['section'] = 'actor'
    post_data['period_start'] = '2022-01'
    post_data['gender'] = 'all'
    post_data['page'] = page
    
    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.text, 'html.parser')
    actors = soup.select('li.people_li div.name')
    
    for index, actor in enumerate(actors):
        actor_info_dict = dict()
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_info_dict['이름'] = actor_name
        
        success_num = soup.select('li.people_li ul.num_info li strong')
        success_num_ = success_num[index].text
        actor_info_dict['흥행지수'] = success_num_
        
        mov_list = soup.select('li.people_li ul.mov_list')
        mov_list_ = mov_list[index].select('span')
        actor_info_dict['흥행영화'] = [mov.text for mov in mov_list_]

        ranking = soup.select('li.people_li span.grade')
        ranking_ = ranking[index].text
        actor_info_dict['랭킹'] = ranking_
        
        actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        for actor_detail in actor_details:
            actor_detail_key = actor_detail.select_one('span.tit').text
            actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail))
            actor_detail_value = re.sub('<.*?>', '', actor_detail_value)
            actor_info_dict[actor_detail_key] = actor_detail_value

        actors_info_list.append(actor_info_dict)

actors_info_list

[{'이름': '강하늘',
  '흥행지수': '79,188',
  '흥행영화': ['해적: 도깨비 깃발', '해피 뉴 이어'],
  '랭킹': '1',
  '다른 이름': '김하늘',
  '직업': '배우',
  '생년월일': '1990-02-21',
  '성별': '남',
  '홈페이지': '\nhttp://weibo.com/galpos3?is_hot=1\n',
  '신장/체중': '181cm, 70kg',
  '학교': '중앙대학교 연극학과'},
 {'이름': '한효주',
  '흥행지수': '67,411',
  '흥행영화': ['해적: 도깨비 깃발'],
  '랭킹': '2',
  '직업': '배우',
  '생년월일': '1987-02-22',
  '성별': '여',
  '홈페이지': '\nhttps://www.facebook.com/hhj.official\n',
  '신장/체중': '170cm',
  '학교': '동국대학교 연극영화',
  '취미': '영화감상'},
 {'이름': '이광수',
  '흥행지수': '59,525',
  '흥행영화': ['해적: 도깨비 깃발', '해피 뉴 이어'],
  '랭킹': '3',
  '직업': '배우',
  '생년월일': '1985-07-14',
  '성별': '남',
  '홈페이지': '\nhttps://twitter.com/masijacoke85\nhttps://www.instagram.com/masijacoke850714/\n',
  '신장/체중': '190cm',
  '소속사': '킹콩엔터테인먼트'},
 {'이름': '권상우',
  '흥행지수': '48,151',
  '흥행영화': ['해적: 도깨비 깃발'],
  '랭킹': '4',
  '다른 이름': 'Kwon Sang Woo',
  '직업': '배우',
  '생년월일': '1976-08-05',
  '성별': '남',
  '신장/체중': '183cm, 72kg',
  '학교': '한남대학교 미술교육학 학사',
  '취미': '수영, 헬스, 복싱',
  '특기':